In [2]:
import os
import re
import dotenv
import numpy as np
from numpy import inf
import pandas as pd
from pprint import pprint

In [133]:
# Load environment variables
project_dir = os.path.join(os.path.abspath(''), os.pardir)
dotenv_path = os.path.join(project_dir, '.env')

dotenv.load_dotenv(dotenv_path)

True

In [6]:
df = pd.read_csv('/home/jovyan/git_projects/machine-learning-project/data/processed/daily_measures_features_full.csv', encoding='utf-8')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4168 entries, 0 to 4167
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Message Text               4168 non-null   object 
 1   Date                       4168 non-null   object 
 2   Impressions                4168 non-null   float64
 3   Comments                   381 non-null    float64
 4   Likes                      2364 non-null   float64
 5   Shares                     886 non-null    float64
 6   Other Engagements          2361 non-null   float64
 7   Engagements                2748 non-null   float64
 8   days_since_posted          4168 non-null   float64
 9   fanpage_id                 4168 non-null   float64
 10  ID                         4168 non-null   object 
 11  has_any_image              4168 non-null   bool   
 12  hashtags                   3329 non-null   object 
 13  has_any_hashtag            4168 non-null   bool 

In [10]:
# aggregate everything to sum (leistungsdaten) in order to get 1 row per ID
df_agg = df.groupby(['ID']).agg({
    'Impressions': 'sum',
    'Likes': 'sum',
    'Shares': 'sum',
    'Comments': 'sum',
    'Other Engagements': 'sum',
    'days_since_posted': 'max', # basically as feature more -> better, post acknoledge or good for LI algorithms
    'Engagements': 'sum',
    'Date': 'first',
    'fanpage_id': 'first',
    'has_any_image': 'first',
    'hashtags': 'first',
    'has_any_hashtag': 'first',
    'hour_posted': 'first',
    'dow_posted': 'first',
    'post_len': 'first',
    'timestamp_posted': 'first',
    'Dominant_Topic': 'first',
    'text_clean_and_translated': 'first',
    'Topic_Perc_Contrib': 'first'
}).reset_index()

In [26]:
df_agg = df_agg.rename(columns={
    'days_since_posted': 'days_since_last_collected_data',
    'Impressions': 'sum_imps',
    'Engagements': 'sum_engs',
    'Likes': 'sum_likes',
    'Shares': 'sum_shares',
    'Comments': 'sum_comments',
    'Other Engagements': 'sum_engs_other',
    'Dominant_Topic': 'topic_lda',
    'Topic_Perc_Contrib': 'sure_belongs_to_topic',
    'text_clean_and_translated': 'text_posted',
    'post_len': 'length_text_posted'
})

In [27]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4168 entries, urn:li:share:7015109880122384384 to urn:li:ugcPost:7091797022185295873
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   sum_imps                        4168 non-null   float64       
 1   sum_likes                       4168 non-null   float64       
 2   sum_shares                      4168 non-null   float64       
 3   sum_comments                    4168 non-null   float64       
 4   sum_engs_other                  4168 non-null   float64       
 5   days_since_last_collected_data  4168 non-null   float64       
 6   sum_engs                        4168 non-null   float64       
 7   Date                            4168 non-null   object        
 8   fanpage_id                      4168 non-null   float64       
 9   has_any_image                   4168 non-null   bool          
 10  hashtags        

In [81]:
df_agg['timestamp_posted'] = pd.to_datetime(df_agg['timestamp_posted'])
df_agg['week_posted'] = df_agg['timestamp_posted'].dt.isocalendar().week
df_agg['minute_posted'] = df_agg['timestamp_posted'].dt.minute
df_agg['day_posted'] = df_agg['timestamp_posted'].dt.day

In [56]:
df_agg['topic_lda'] = df_agg['topic_lda'].astype(str)

In [ ]:
df_agg = df_agg.set_index('ID')

In [39]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4168 entries, urn:li:share:7015109880122384384 to urn:li:ugcPost:7091797022185295873
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   sum_imps                        4168 non-null   float64       
 1   sum_likes                       4168 non-null   float64       
 2   sum_shares                      4168 non-null   float64       
 3   sum_comments                    4168 non-null   float64       
 4   sum_engs_other                  4168 non-null   float64       
 5   days_since_last_collected_data  4168 non-null   float64       
 6   sum_engs                        4168 non-null   float64       
 7   Date                            4168 non-null   object        
 8   fanpage_id                      4168 non-null   float64       
 9   has_any_image                   4168 non-null   bool          
 10  hashtags        

In [38]:
from keybert import KeyBERT

kw_model = KeyBERT()
extract_keywords = lambda doc: kw_model.extract_keywords(doc)

In [41]:
df_agg['bert_keywords'] = df_agg.text_posted.apply(extract_keywords)

In [48]:
df_agg['bert_keywords'] = df_agg.bert_keywords.apply(lambda x: ','.join([j[0] for j in x]))

In [49]:
df_agg['eng_rate'] = df_agg.sum_engs / df_agg.sum_imps

In [50]:
df_agg

,sum_imps,sum_likes,sum_shares,sum_comments,sum_engs_other,days_since_last_collected_data,sum_engs,Date,fanpage_id,has_any_image,...,dow_posted,length_text_posted,timestamp_posted,topic_lda,text_posted,sure_belongs_to_topic,week_posted,minute_posted,bert_keywords,eng_rate
ID,,,,,,,,,,,,,,,,,,,,,
urn:li:share:7015109880122384384,3.0,0.0,0.0,0.0,0.0,73.0,0.0,2023-03-15 00:00:00,154517.0,False,...,Sunday,213,2023-01-01 01:22:10.005,0,We wish you all a happy and healthy 2023. Tack...,0.7553,52,22,"2023,wishes,wish,healthy,happy",0.000000
urn:li:share:7015180072244367361,11.0,0.0,0.0,0.0,0.0,5.0,0.0,2023-01-06 00:00:00,50.0,True,...,Sunday,266,2023-01-01 06:01:05.182,0,happy new year from everyone at bosch and we h...,0.8158,52,1,"bosch,2023,success,year,innovative",0.000000
urn:li:share:7015210047886524416,17119.0,387.0,16.0,1.0,248.0,1.0,652.0,2023-01-02 00:00:00,118.0,True,...,Sunday,330,2023-01-01 08:00:11.861,4,are you ready for 2023 we are as we welcome th...,0.7938,52,0,"2023,future,innovations,sustainable,groundbrea...",0.038086
urn:li:share:7015240414274158592,376.0,10.0,0.0,0.0,4.0,5.0,14.0,2023-01-06 00:00:00,2165.0,True,...,Sunday,189,2023-01-01 10:00:51.753,0,we at krones hope you had a wonderful new year...,0.7359,52,0,"krones,2023,year,eve,hope",0.037234
urn:li:share:7015249631085240320,2792.0,146.0,13.0,0.0,46.0,0.0,205.0,2023-01-01 00:00:00,138174.0,True,...,Sunday,29,2023-01-01 10:37:29.286,0,happy new year to all of you,0.6477,52,37,"happy,year,new",0.073424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
urn:li:ugcPost:7090716644628381696,41225.0,132.0,5.0,0.0,8984.0,0.0,9121.0,2023-07-31 00:00:00,113.0,False,...,Monday,844,2023-07-28 17:36:27.418,0,our board member olivier is a graduate of fran...,0.8494,30,36,"banking,banks,risk,hausbank,safeguard",0.221249
urn:li:ugcPost:7090722993714122753,1868.0,20.0,3.0,0.0,107.0,0.0,130.0,2023-07-28 00:00:00,113.0,False,...,Friday,732,2023-07-28 18:01:41.210,0,inclusivity is at the heart of our culture. th...,0.8447,30,1,"inclusive,hearing,impairment,inclusivity,inclu...",0.069593
urn:li:ugcPost:7091713157756248064,14910.0,284.0,9.0,8.0,1252.0,0.0,1553.0,2023-07-31 00:00:00,2165.0,False,...,Monday,655,2023-07-31 11:36:14.639,4,are you ready to produce preforms and caps ind...,0.8314,31,36,"moulding,preforms,plastics,caps,injection",0.104158


In [82]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4168 entries, urn:li:share:7015109880122384384 to urn:li:ugcPost:7091797022185295873
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   sum_imps                        4168 non-null   float64       
 1   sum_likes                       4168 non-null   float64       
 2   sum_shares                      4168 non-null   float64       
 3   sum_comments                    4168 non-null   float64       
 4   sum_engs_other                  4168 non-null   float64       
 5   days_since_last_collected_data  4168 non-null   float64       
 6   sum_engs                        4168 non-null   float64       
 7   Date                            4168 non-null   object        
 8   fanpage_id                      4168 non-null   float64       
 9   has_any_image                   4168 non-null   bool          
 10  hashtags        

In [97]:
X = df_agg.loc[:, [
    'has_any_image',
    'hashtags',
    'has_any_hashtag',
    'hour_posted',
    'day_posted',
    'fanpage_id',
    'timestamp_posted',
    'length_text_posted',
    'topic_lda',
    'sure_belongs_to_topic',
    'week_posted',
    'minute_posted',
    'bert_keywords']
]
y = df_agg['eng_rate']

In [98]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4168 entries, urn:li:share:7015109880122384384 to urn:li:ugcPost:7091797022185295873
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   has_any_image          4168 non-null   bool          
 1   hashtags               3329 non-null   object        
 2   has_any_hashtag        4168 non-null   bool          
 3   hour_posted            4168 non-null   int64         
 4   day_posted             4168 non-null   int64         
 5   fanpage_id             4168 non-null   float64       
 6   timestamp_posted       4168 non-null   datetime64[ns]
 7   length_text_posted     4168 non-null   int64         
 8   topic_lda              4168 non-null   object        
 9   sure_belongs_to_topic  4168 non-null   float64       
 10  week_posted            4168 non-null   UInt32        
 11  minute_posted          4168 non-null   int64         
 12  bert_k

In [99]:
# One hot encoding for all categorical variables (decision tree needed)
X_dt = X.copy()

In [100]:
import unicodedata as ud
X_dt = pd.concat([
   X_dt,
   X_dt.topic_lda.str.get_dummies().add_prefix(f'lda_topic_'),
   pd.get_dummies(X_dt.bert_keywords.str.split(',').explode()).groupby(level=0).sum(),
   pd.get_dummies(X_dt.hashtags.apply(lambda x: ud.normalize('NFKD', str(x))).str.split(',').explode()).groupby(level=0).sum()
], axis=1).drop(columns=['hashtags', 'topic_lda', 'bert_keywords'])

In [101]:
X_dt = X_dt.groupby(level=0).sum()

In [102]:
X_dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4168 entries, urn:li:share:7015109880122384384 to urn:li:ugcPost:7091797022185295873
Columns: 10183 entries, has_any_image to điện
dtypes: UInt32(1), float64(2), int64(12), uint8(10168)
memory usage: 40.9+ MB


In [103]:
X_dt

,has_any_image,has_any_hashtag,hour_posted,day_posted,fanpage_id,length_text_posted,sure_belongs_to_topic,week_posted,minute_posted,lda_topic_0,...,zmiana,zniżka,zrownowazony,zrównoważony,zrównoważonym,zrównoważonyrozwój,zukunft,zukunftit,zwiekszenie,điện
ID,,,,,,,,,,,,,,,,,,,,,
urn:li:share:7015109880122384384,0,1,1,1,154517.0,213,0.7553,52,22,1,...,0,0,0,0,0,0,0,0,0,0
urn:li:share:7015180072244367361,1,1,6,1,50.0,266,0.8158,52,1,1,...,0,0,0,0,0,0,0,0,0,0
urn:li:share:7015210047886524416,1,0,8,1,118.0,330,0.7938,52,0,0,...,0,0,0,0,0,0,0,0,0,0
urn:li:share:7015240414274158592,1,0,10,1,2165.0,189,0.7359,52,0,1,...,0,0,0,0,0,0,0,0,0,0
urn:li:share:7015249631085240320,1,0,10,1,138174.0,29,0.6477,52,37,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
urn:li:ugcPost:7090716644628381696,0,1,17,28,113.0,844,0.8494,30,36,1,...,0,0,0,0,0,0,0,0,0,0
urn:li:ugcPost:7090722993714122753,0,1,18,28,113.0,732,0.8447,30,1,1,...,0,0,0,0,0,0,0,0,0,0
urn:li:ugcPost:7091713157756248064,0,1,11,31,2165.0,655,0.8314,31,36,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
y.describe()

count    4168.000000
mean        0.060382
std         0.125576
min         0.000000
25%         0.000000
50%         0.027567
75%         0.066917
max         2.333333
Name: eng_rate, dtype: float64

In [105]:
X.to_csv('/home/jovyan/git_projects/machine-learning-project/data/processed/X_catboost.csv', encoding='utf-8', index=True)
X_dt.to_csv('/home/jovyan/git_projects/machine-learning-project/data/processed/X_decision_trees.csv', encoding='utf-8', index=True)
y.to_csv('/home/jovyan/git_projects/machine-learning-project/data/processed/y_both.csv', encoding='utf-8', index=True)